In [3]:
import time
import datetime
import math

!echo "CREATE DATABASE izstreaming" | psql --u postgres

starttime ='2019-09-03 15:00:00'
endtime ='2019-10-03 16:00:00'
batch = 5

starttimestamp = time.mktime(time.strptime(starttime, '%Y-%m-%d %H:%M:%S'))
endtimestamp = time.mktime(time.strptime(endtime, '%Y-%m-%d %H:%M:%S'))

ERROR:  database "izstreaming" already exists


In [52]:
timegap = math.floor((endtimestamp-starttimestamp)/batch)
i=timegap
start=""
x=customparksession.sql("select customerid,transaction_complete_dtm,payment_type,total_amount,other_amount from testdb.ds_cc_transaction where transaction_complete_dtm>='{}' and transaction_complete_dtm<='{}'".format(starttime,endtime))
while i<=(endtimestamp-starttimestamp):
    if start=="":start=starttime
    else:start=end
    datetime_struct = datetime.datetime.fromtimestamp(starttimestamp)
    datetime_obj = (datetime_struct + datetime.timedelta(seconds=i))
    end = datetime_obj.strftime('%Y-%m-%d %H:%M:%S')
    data=x.where("transaction_complete_dtm>='%s' and transaction_complete_dtm<'%s'" %(start,end)).coalesce(200)
    rounds=int(i/timegap)
    print(f"*******************{rounds} th round:{data.count()}*******************")
    print (start+"~" +end)
    
    mode = "overwrite"
    #mode = "append"
    url = "jdbc:postgresql://101.15.32.2:5432/izstreaming"
    properties = {"user": "postgres","password": "postgres","driver": "org.postgresql.Driver"}
    x.write.jdbc(url=url, table="testiz", mode=mode, properties=properties)
    print(f"************************write data succeed!***************************")
    i=i+timegap
    time.sleep(5)

*******************1 th round:3432*******************
2019-09-03 15:00:00~2019-09-03 15:20:00
************************write data succeed!***************************
*******************2 th round:3540*******************
2019-09-03 15:20:00~2019-09-03 15:40:00
************************write data succeed!***************************
*******************3 th round:3419*******************
2019-09-03 15:40:00~2019-09-03 16:00:00
************************write data succeed!***************************


In [17]:
!echo "select * from testiz limit 5" | psql --u postgres izstreaming

 customerid | transaction_complete_dtm | payment_type  | total_amount | other_amount 
----------+---------------------+-----------+------------+-------
 JE0E+z04 | 2019-09-03 15:21:05 | ALIPAY    |       1000 |  1000
 +H0$!$4z | 2019-09-03 15:28:02 | WECHAT |        500 |   500
 +zHbJzJJ | 2019-09-03 15:50:37 | ALIPAY    |       1000 |  1000
 4+H!0+4! | 2019-09-03 15:11:00 | ALIPAY    |       1000 |  1000
 +Hb4EzJz | 2019-09-03 15:08:12 | WECHAT |        500 |   500
(5 rows)

